In [4]:
%pip install -q -U keras-tuner
%pip install -q -U tensorboard-plugin-profile

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

## Data Fetching

In [6]:
from utils.data_fetch import read_data
data = read_data()

C:\Users\Vardan\AppData\Local\Temp\ipykernel_5676\1164869249.py:2: DeprecationWarning: [Deprecated][in version 6.0.0]: client_factory will be replaced by gspread.http_client types
  data = read_data()


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Spindle Speed     27 non-null     float64
 1   Feed (mm/min)     27 non-null     float64
 2   Tool Dia. (mm)    27 non-null     float64
 3   MRR (mm^3 / min)  27 non-null     float64
 4   L_e               27 non-null     float64
dtypes: float64(5)
memory usage: 1.2 KB


## Data Preprocessing

In [8]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(data, test_size=0.2, random_state=32)

In [9]:
X_train_full = train_set.drop(["L_e", "MRR (mm^3 / min)"], axis = 1)
y_train_full =pd.concat([train_set["L_e"], train_set["MRR (mm^3 / min)"]], axis=1)

In [10]:
train_set, val_set = train_test_split(train_set, test_size=0.2, random_state=32)

In [11]:
X_train = train_set.drop(["L_e", "MRR (mm^3 / min)"], axis = 1)
y_train=pd.concat([train_set["L_e"], train_set["MRR (mm^3 / min)"]], axis=1)

In [12]:
X_val = val_set.drop(["L_e", "MRR (mm^3 / min)"], axis = 1)
y_val=pd.concat([val_set["L_e"], val_set["MRR (mm^3 / min)"]], axis=1)

In [13]:
X_test = test_set.drop(["L_e", "MRR (mm^3 / min)"], axis = 1)
y_test=pd.concat([test_set["L_e"], test_set["MRR (mm^3 / min)"]], axis=1)

In [29]:
X_train.shape

(16, 3)

In [30]:
norm_layer = keras.layers.Normalization(input_shape=X_train.shape[1:])

In [31]:
model = keras.models.Sequential([
    norm_layer,
    keras.layers.BatchNormalization(),
    keras.layers.Dense(500, activation="relu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="relu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    keras.layers.Dense(2)
])
model.compile(loss="mse", optimizer=keras.optimizers.Adam(learning_rate=1e-3), metrics=["RootMeanSquaredError", "mse"])
norm_layer.adapt(X_train)

In [32]:
X_train.shape

(16, 3)

In [33]:
y_train.shape

(16, 2)

In [34]:
X_val.shape

(5, 3)

In [35]:
y_val.shape

(5, 2)

In [36]:
from pathlib import Path
from time import strftime
def get_run_logdir(root_logdir="my_logs"):
 return Path(root_logdir) / strftime("run_%Y_%m_%d_%H_%M_%S")
run_logdir = get_run_logdir() 

In [37]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)

In [38]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normaliza  (None, 3)                 7         
 tion)                                                           
                                                                 
 batch_normalization_6 (Bat  (None, 3)                 12        
 chNormalization)                                                
                                                                 
 dense_8 (Dense)             (None, 500)               2000      
                                                                 
 batch_normalization_7 (Bat  (None, 500)               2000      
 chNormalization)                                                
                                                                 
 dense_9 (Dense)             (None, 300)               150300    
                                                      

In [39]:
history = model.fit(X_train,y_train, epochs=500, verbose=1, callbacks=[ early_stopping_cb],validation_data=(X_val,y_val))

Epoch 1/500
1/1 [==============================] - 3s 3s/step - loss: 30.1543 - root_mean_squared_error: 5.4913 - mse: 30.1543 - val_loss: 36.8371 - val_root_mean_squared_error: 6.0694 - val_mse: 36.8371
Epoch 2/500
1/1 [==============================] - 0s 104ms/step - loss: 15.5275 - root_mean_squared_error: 3.9405 - mse: 15.5275 - val_loss: 32.3912 - val_root_mean_squared_error: 5.6913 - val_mse: 32.3912
Epoch 3/500
1/1 [==============================] - 0s 62ms/step - loss: 9.2701 - root_mean_squared_error: 3.0447 - mse: 9.2701 - val_loss: 28.6053 - val_root_mean_squared_error: 5.3484 - val_mse: 28.6053
Epoch 4/500
1/1 [==============================] - 0s 67ms/step - loss: 6.0432 - root_mean_squared_error: 2.4583 - mse: 6.0432 - val_loss: 25.5555 - val_root_mean_squared_error: 5.0552 - val_mse: 25.5555
Epoch 5/500
1/1 [==============================] - 0s 66ms/step - loss: 3.8712 - root_mean_squared_error: 1.9675 - mse: 3.8712 - val_loss: 23.3684 - val_root_mean_squared_error: 4.8

In [43]:
rmse_score = model.evaluate(X_test,y_test)

1/1 [==============================] - 0s 51ms/step - loss: 4.3511 - root_mean_squared_error: 2.0859 - mse: 4.3511


In [44]:
X_new = X_train[:3]

In [45]:
X_train[:3]

,Spindle Speed,Feed (mm/min),Tool Dia. (mm)
1,3000.0,10.0,2.0
16,4000.0,20.0,2.0
7,3000.0,20.0,2.0


In [47]:
model.predict(X_new)

1/1 [==============================] - 0s 23ms/step


array([[ 2.5970462e-01,  3.0021169e+00],
       [ 1.4138399e-03,  6.3249488e+00],
       [-1.1713730e-01,  4.6781077e+00]], dtype=float32)

In [49]:
y_train[:3]

,L_e,MRR (mm^3 / min)
1,0.32,3.04
16,0.09,6.53
7,0.01,4.75


### Model Tuning

In [50]:
def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons = hp.Int("n_neurons", min_value=100, max_value=500)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2,sampling="log")
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model = keras.Sequential()
    model.add(norm_layer)
    for _ in range(n_hidden):
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Dense(n_neurons, activation="relu", kernel_initializer="he_normal"))
    model.add(keras.layers.Dense(2))
    model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])
    return model

In [51]:
tuner = kt.RandomSearch(
 build_model, objective=kt.Objective("val_root_mean_squared_error", direction="min"), max_trials=10, overwrite=True,
 directory="Model_Tuning", project_name="my_rnd_search", seed=42)

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)

tuner.search(X_train, y_train, epochs=150, validation_data=(X_val, y_val), callbacks=[early_stop])

Trial 10 Complete [00h 01m 34s]
val_root_mean_squared_error: 4.7236528396606445

Best val_root_mean_squared_error So Far: 3.799974203109741
Total elapsed time: 00h 06m 26s


In [52]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [53]:
best_trial = tuner.oracle.get_best_trials(num_trials=1)[0]

In [54]:
best_trial.summary()

Trial 01 summary
Hyperparameters:
n_hidden: 2
n_neurons: 456
learning_rate: 0.000505466655750672
Score: 3.799974203109741


In [55]:
Final_model = tuner.get_best_models(num_models=1)[0]

In [56]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir, profile_batch=(100, 200))

In [57]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

In [58]:
Final_model.fit(X_train_full, y_train_full, epochs=200, callbacks=[tensorboard_cb, early_stopping_cb])

Epoch 1/200
1/1 [==============================] - 2s 2s/step - loss: 2.8542 - root_mean_squared_error: 1.6894
Epoch 2/200
1/1 [==============================] - 0s 35ms/step - loss: 2.4789 - root_mean_squared_error: 1.5744
Epoch 3/200
1/1 [==============================] - 0s 36ms/step - loss: 2.1227 - root_mean_squared_error: 1.4569
Epoch 4/200
1/1 [==============================] - 0s 32ms/step - loss: 1.9374 - root_mean_squared_error: 1.3919
Epoch 5/200
1/1 [==============================] - 0s 31ms/step - loss: 1.7996 - root_mean_squared_error: 1.3415
Epoch 6/200
1/1 [==============================] - 0s 38ms/step - loss: 1.5728 - root_mean_squared_error: 1.2541
Epoch 7/200
1/1 [==============================] - 0s 29ms/step - loss: 1.2682 - root_mean_squared_error: 1.1262
Epoch 8/200
1/1 [==============================] - 0s 34ms/step - loss: 0.9973 - root_mean_squared_error: 0.9986
Epoch 9/200
1/1 [==============================] - 0s 37ms/step - loss: 0.8402 - root_mean_squared

In [59]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs

Reusing TensorBoard on port 6006 (pid 9016), started 4 days, 7:55:55 ago. (Use '!kill 9016' to kill it.)

In [60]:
X_new

,Spindle Speed,Feed (mm/min),Tool Dia. (mm)
1,3000.0,10.0,2.0
16,4000.0,20.0,2.0
7,3000.0,20.0,2.0


In [61]:
Final_model.predict(X_new)

1/1 [==============================] - 0s 275ms/step


array([[0.34725618, 2.8428962 ],
       [0.07646406, 6.0699844 ],
       [0.06207085, 4.4208775 ]], dtype=float32)

In [62]:
y_train[:3]

,L_e,MRR (mm^3 / min)
1,0.32,3.04
16,0.09,6.53
7,0.01,4.75


In [1]:
Final_model.save("Final_neural_network.h5")

NameError: name 'Final_model' is not defined

In [14]:
model = tf.keras.models.load_model('Final_neural_network.h5')

In [15]:
model.evaluate(X_train, y_train)


1/1 [==============================] - 1s 850ms/step - loss: 0.0925 - root_mean_squared_error: 0.3041


[0.09247415512800217, 0.30409565567970276]

In [16]:
model.evaluate(X_test,y_test)

1/1 [==============================] - 0s 109ms/step - loss: 1.2252 - root_mean_squared_error: 1.1069


[1.2251598834991455, 1.1068694591522217]

In [17]:
y_pred = model.predict(X_test)

1/1 [==============================] - 0s 495ms/step


In [18]:
y_pred

array([[-0.2489668 ,  3.8575652 ],
       [ 1.705666  ,  7.2124214 ],
       [ 0.38264185, -0.1564873 ],
       [ 0.7502742 ,  0.38481128],
       [ 0.52213395,  1.6359488 ],
       [-0.03109323,  2.060239  ]], dtype=float32)

In [19]:
y_test

,L_e,MRR (mm^3 / min)
18,0.26,0.96
26,0.33,6.86
12,0.36,1.12
0,0.05,1.08
6,0.05,0.81
21,0.10,1.35


In [20]:
from ann_visualizer.visualize import ann_viz

# Visualize the saved Keras model
ann_viz(model, view=True)

ModuleNotFoundError: No module named 'ann_visualizer'